# Chains in SK

## Outline

* Sequential Chains with kernel.run_async


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [6]:
df.Review[5]

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

## LLM Chain

In [8]:
import semantic_kernel as sk
import os
import logging
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
# Demonstrate some basic logging to debug the chain
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('__name__')
kernel=sk.Kernel(log=logger)
api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
)

In [9]:
prompt = "What is the best name to describe \
    a company that makes {{ $input }} ?"


In [10]:
# Create a semantic function based on prompt to name a company based on product it makes. 
# Make it creative by setting a high temperature for the LLM
productnamer = kernel.create_semantic_function(prompt, temperature=0.9)

In [11]:
product = "Queen Size Sheet Set"
answer = await kernel.run_async(productnamer, input_str=product)
print(answer)

Some potential names to describe a company that makes Queen Size Sheet Sets could include:

1. "Royal Rest Linens"
2. "Queen Comfort Co."
3. "Regal Bedding Solutions"
4. "Sleepy Monarch Linens"
5. "Crown Jewel Sheets"
6. "Majestic Dreams Bedding"
7. "Imperial Sleep Essentials"
8. "Quintessential Queen Linens"
9. "Palatial Bedding Collections"
10. "Supreme Queen Beddings"


## SimpleSequentialChain

In [12]:
# Now lets chain two skills together sequerntially with output of first passed as input to second prompt
# prompt template 1
first_prompt = "What is the best name to describe \
    a company that makes {{ $input }}?"

# Semantic Function 1
func_one = kernel.create_semantic_function(first_prompt, temperature=0.9)

In [13]:

# prompt template 2
second_prompt = "Write a 20 words description for the following \
    company:{{ $input }}"

# chain 2
func_two = kernel.create_semantic_function(second_prompt, temperature=0.9)

In [14]:
answer = await kernel.run_async(func_one, func_two, input_str="Queen Size Sheet Set")
print(answer)


The Royal Sleep Company provides luxurious and comfortable sleep solutions with high-quality mattresses and bedding for a restful night's sleep.


## Sequential Chain

Next we will look at more flexible ways to chain together semantic functions beyond a sequential pipeline. Here you need to save output of a step into context variables using Semantic Kernel's **Native Functions** which are registered as semantic functions. Currently you need to call these native semantic functions after an LLM function to store output to a specific context variable so it can be used in latter LLM functions. 

In [15]:
from semantic_kernel.skill_definition import sk_function
from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter
from semantic_kernel import SKContext

class CopyContext:
    """
    Description: Copies context to do chaining
    """

    @sk_function(
        description="Copies input to another context variable"
    )
    @sk_function_context_parameter(name="English_Review", description="Review in English.")
    def EnglishReview(self, context: SKContext) -> str:
        context["English_Review"] = context["input"]
        return context["input"]


    @sk_function(
        description="Copies input to another context variable"
    )
    @sk_function_context_parameter(name="summary", description="Review in English.")
    def Summary(self, context: SKContext) -> str:
        context["summary"] = context["input"]
        return context["input"]
    
    @sk_function(
        description="Copies input to another context variable"
    )
    @sk_function_context_parameter(name="language", description="Review in English.")
    def Language(self, context: SKContext) -> str:
        context["language"] = context["input"]
        return context["input"]
    
    

In [16]:
copy_context_skill = kernel.import_skill(CopyContext())
v_englishreview = copy_context_skill["EnglishReview"]
v_language = copy_context_skill["Language"]
v_summary = copy_context_skill["Summary"]

In [17]:
review=df.Review[5]
context_variables = sk.ContextVariables()
context_variables["review"] = review

In [18]:
# prompt template 1: translate to english
first_prompt = """Translate the following review to english:
    {{ $review }}"""

# chain 1: input= Review and output= English_Review
# Semantic Function 1
func_one = kernel.create_semantic_function(first_prompt, temperature=0.9)


In [19]:
second_prompt = """Can you summarize the following review in 1 sentence:
    {{ $English_Review }}"""

# chain 2: input= English_Review and output= summary
func_two = kernel.create_semantic_function(second_prompt, temperature=0.9)

In [20]:
# Lets try to first chain two functions before trying a more complex chain
# Chain looks like this: func_one-> store output in context -> func_two
answer = await kernel.run_async(func_one, v_englishreview,func_two, input_vars=context_variables)
print(answer)

The reviewer believes the taste of the product is average, with disappointing foam quality, and suspects a potential issue with the batch or its authenticity.


In [21]:
# Now lets chain more functions together in more flexible manner
# prompt template 3: translate to english
third_prompt = "What language is the following review:\n\n{{ $review }}"

# chain 3: input= Review and output= language
func_three = kernel.create_semantic_function(third_prompt, temperature=0.9)

In [22]:
# prompt template 4: follow up message
fourth_prompt = """Write a follow up response to the following 
    summary in the specified language:
    Summary: {{ $summary }}\n\nLanguage: {{ $language }}"""

# chain 4: input= summary, language and output= followup_message
func_four = kernel.create_semantic_function(fourth_prompt, temperature=0.9)

Now call a more elaborate chain where output from a step is used as input not just in next step
Chain looks like this:
<pre>
                                         +-----------------------------
                                         |                            |
                                         |                            V
 (Input)Review ----> func_one   --->  func_two  func_three  ----->  func_four
          |                                        ^
          |                                        |
          +-----------------------------------------
</pre>
   
Notice that after each function , we have to call a native function (like v_englighreview, v_summary, v_language) to store output in context so it can be referenced in a subsequent step. In kernel.run_async you can specify a list of functions (semantic or native function) to call which it does in sequence. 

In [23]:
answer = await kernel.run_async(func_one, v_englishreview,func_two, v_summary, func_three, v_language, func_four, input_vars=context_variables)

print(answer)

Cher(e) client(e),

Nous vous remercions d'avoir partagé votre avis sur notre produit. Nous sommes désolés d'apprendre que vous êtes déçu(e) par son goût, en particulier par le fait que la mousse ne tienne pas et que vous suspectiez d'avoir reçu un lot ancien ou une contrefaçon.

Nous tenons à vous assurer que tous nos produits sont soigneusement contrôlés avant d'être commercialisés. Cependant, il est possible que des variations de qualité puissent se produire de temps en temps, et nous en sommes sincèrement désolés. Votre commentaire a été transmis à notre équipe responsable de la production afin qu'elle puisse enquêter sur cette situation.

Nous aimerions avoir plus de détails sur votre expérience pour mieux comprendre ce qui s'est passé. Pourriez-vous nous fournir le numéro de lot ou toute autre information pertinente figurant sur l'emballage du produit ? Cela nous aiderait grandement dans notre investigation.

En tant qu'entreprise soucieuse de la satisfaction de nos clients, nous